# 题目

> 给定一组 n 人（编号为 1, 2, ..., n）， 我们想把每个人分进任意大小的两组。每个人都可能不喜欢其他人，那么他们不应该属于同一组。  
给定整数 n 和数组 dislikes ，其中 dislikes[i] = [ai, bi] ，表示不允许将编号为 ai 和 bi的人归入同一组。当可以用这种方法将所有人分进两组时，返回 true；否则返回 false。

# 方法一：并查集

> 有关并查集： https://zhuanlan.zhihu.com/p/93647900 。  
> 由于最后只有两组，所以某一个人全部不喜欢人一定会在同一个组中，我们可以用「并查集」进行连接，并判断这个人是否与他不喜欢的人相连，如果相连则表示存在冲突，否则说明此人和他不喜欢的人在当前可以进行合法分组。

## 复杂度

- 时间复杂度: $O(n+m\alpha(n))$ ，其中 $n$ 是给定的人数， $m$ 为给定的 dislike 数组的大小， $\alpha$ 是反 Ackerman 函数。

- 空间复杂度: $O(n+m)$ ，其中 $n$ 是给定的人数， $m$ 为给定的 dislike 数组的大小。

## 代码

In [1]:
# 构建并查集
class UnionFind:
    def __init__(self, n):
        self.fa = list(range(n))  # 用于记录每个元素的值及其指向的值。第x个位置表示该元素值为x，其对应的值fa[x]表示元素x指向元素fa[x]
        self.rank = [1] * n  # 用于记录某个代表元素对应的集合的复杂度（节点个数）
    
    # 找到某个集合的代表元素，并进行路径压缩
    def find(self, x):
        
        # 若元素x不指向自己，则寻找x指向的元素fa[x],看其是否指向自己
        # 若fa[fa[x]]仍不指向自己，则继续找，直到找到指向自己的元素（一个集合的代表元素）
        # 找到代表元素后，迭代函数返回该代表元素的值，在回溯过程中，由于最后一个find返回了代表元素的值，因此迭代中的每个fa[x]都会变得指向代表元素
        if self.fa[x] != x:
            self.fa[x] = self.find(self.fa[x])
        
        return self.fa[x]

    # 将两个元素所对应的集合合并
    def union(self, x, y):
        
        # 找到元素x和元素y所在集合的代表元素，并进行路径压缩
        fx, fy = self.find(x), self.find(y)
        
        # 若元素x和元素y所在集合为同一个，返回None
        if fx == fy:
            return
        
        # 将rank更小的集合连接到rank更大的集合
        # 若元素x所在集合的最大路径长度小于元素x所在集合的最大路径长度，则交换fx和fy
        if self.rank[fx] < self.rank[fy]:
            fx, fy = fy, fx  # fx代表x,y中rank更大的集合的代表元素，fy代表x,y中rank更小的集合的代表元素
        
        # 更新合并后集合的代表元素的rank
        self.rank[fx] += self.rank[fy]
        
        # 将较小集合的代表元素指向较大集合的代表元素
        self.fa[fy] = fx

    # 判断两个元素是否属于同一集合
    def is_connected(self, x, y):
        return self.find(x) == self.find(y)

# 解法
class Solution:
    def possibleBipartition(self, n, dislikes):
        # 包含n个列表的列表，位置x对应编号x+1
        g = [[] for _ in range(n)]  # 列表g[x]列举了不与编号x+1同集合的编号
        
        # 遍历dislikes列表，填充列表g
        for x, y in dislikes:
            g[x - 1].append(y - 1)
            g[y - 1].append(x - 1)
        
        # 构建并查集
        uf = UnionFind(n)
        
        for x, nodes in enumerate(g):  # x代表编号x+1，nodes代表不与编号x+1相邻的编号
            # 对每个编号，将与之不相邻的编号放到一个集合里
            for y in nodes:
                uf.union(nodes[0], y)
                # x和y若属于一个集合，则出现矛盾
                if uf.is_connected(x, y):
                    return False
        return True

#### 测试一

In [4]:
n = 4
dislikes = [[1,2],[1,3],[2,4]]

test = Solution()
test.possibleBipartition(n, dislikes)

True

#### 测试二

In [5]:
n = 3
dislikes = [[1,2],[1,3],[2,3]]

test = Solution()
test.possibleBipartition(n, dislikes)

False